In [1]:
import pandas as pd
from IPython.display import display  # Melhor exibição no Jupyter

def carregar_excel(nome_arquivo):
    """Carrega um arquivo Excel e retorna um DataFrame."""
    try:
        df = pd.read_excel(nome_arquivo)
        print(f"Arquivo '{nome_arquivo}' carregado com sucesso!")
        return df
    except FileNotFoundError:
        print(f"Erro: O arquivo '{nome_arquivo}' não foi encontrado.")
    except Exception as e:
        print(f"Erro ao carregar '{nome_arquivo}': {e}")
    return None

def processar_teste_df(teste_df):
    """Realiza o pré-processamento e transformação do DataFrame teste_df."""
    if teste_df is None:
        return None

    # Renomeação de colunas
    colunas_renomeadas = {
        'REF KINGS': 'Referências',
        'DESCRIÇÃO': 'Nome do Produto',
        'ATACADO': 'Valor atacado',
        'VAR': 'Valor varejo',
        'LINHA': 'Coleção',
        'COD. BARRAS': 'Código de Barras'
    }
    teste_df.rename(columns=colunas_renomeadas, inplace=True)

    # Adição de novas colunas com valores padrão
    colunas_adicionais = {
        'Status do Produto': 1,
        'Código Produto - SKU': teste_df['Referências'],
        'Exibir produto na home': 'Não',
        'Peso do Produto': '',
        'Tamanho do Produto': '',
        'Área de Disponibilidade': '{P}',
        'Descrição do Produto': '',
        'Grade ID': 21,
        'Imagens Vínculadas (Sim ou Não)': 'Sim',
        'Multiplicador Padrão': 1,
        'Quantidade Mínima por Pedido': 6,
        'Quantidade Máxima por Pedido': 999999,
        'Forma de Limite': 'Limite por item da grade'
    }
    for coluna, valor in colunas_adicionais.items():
        teste_df[coluna] = valor

    # Ajuste no formato da coluna 'Referências'
    teste_df['Referências'] = teste_df['Referências'].str.slice(0, -4)

    return teste_df

def realizar_merges(teste_df, cor_df, fornecedores_df, categorias_df, marcas_df):
    """Realiza os merges com os DataFrames auxiliares e renomeia colunas."""
    if teste_df is None:
        return None

    try:
        teste_df = teste_df.merge(cor_df, on='COR').rename(columns={'ID': 'Cor'})
        teste_df = teste_df.merge(fornecedores_df, on='FORNECEDOR').rename(columns={'ID': 'Fornecedor ID'})
        teste_df = teste_df.merge(categorias_df, on='CATEGORIA').rename(columns={'ID': 'Categoria ID'})
        teste_df = teste_df.merge(marcas_df, on='MARCA').rename(columns={'ID': 'Marca ID'})
        
        # Convertendo tipos de dados
        teste_df['Marca ID'] = teste_df['Marca ID'].astype(int)
        teste_df['Fornecedor ID'] = teste_df['Fornecedor ID'].astype(int)

        return teste_df
    except KeyError as e:
        print(f"Erro ao realizar merge: coluna ausente - {e}")
        return None

def organizar_colunas(teste_df):
    """Reorganiza as colunas do DataFrame na ordem desejada e remove duplicatas."""
    if teste_df is None:
        return None

    # Removendo duplicatas com base no SKU
    teste_df = teste_df.drop_duplicates(subset=['Código Produto - SKU'])

    # Definindo a nova ordem das colunas
    colunas_ordenadas = [
        'Referências', 'Código Produto - SKU', 'Status do Produto', 'Nome do Produto', 'Categoria ID',
        'Grade ID', 'Exibir produto na home', 'Valor atacado', 'Peso do Produto', 'Tamanho do Produto',
        'Marca ID', 'Fornecedor ID', 'Código de Barras', 'Coleção', 'Valor varejo', 'Área de Disponibilidade',
        'Descrição do Produto', 'Cor', 'Imagens Vínculadas (Sim ou Não)', 'Multiplicador Padrão',
        'Quantidade Mínima por Pedido', 'Quantidade Máxima por Pedido', 'Forma de Limite'
    ]

    return teste_df[colunas_ordenadas]

def salvar_excel(df, nome_arquivo):
    """Salva o DataFrame em um arquivo Excel."""
    if df is not None:
        df.to_excel(nome_arquivo, index=False)
        print(f"Arquivo salvo como '{nome_arquivo}' com sucesso!")

# --- Execução do script ---

# Carregando os arquivos
teste_df = carregar_excel("teste.xlsx")
cor_df = carregar_excel("cor.xlsx")
categorias_df = carregar_excel("categorias.xlsx")
marcas_df = carregar_excel("marcas.xlsx")
fornecedores_df = carregar_excel("fornecedores.xlsx")

# Processando o DataFrame teste_df
teste_df = processar_teste_df(teste_df)

# Realizando os merges
teste_df = realizar_merges(teste_df, cor_df, fornecedores_df, categorias_df, marcas_df)

# Organizando colunas
teste_df = organizar_colunas(teste_df)

# Exibindo os tipos de dados
if teste_df is not None:
    print("Tipos de dados das colunas:\n", teste_df.dtypes)

# Exibindo a amostra dos dados formatados no Jupyter Notebook
if teste_df is not None:
    display(teste_df.head())

# Salvando o arquivo final
salvar_excel(teste_df, "upload.xlsx")


Arquivo 'teste.xlsx' carregado com sucesso!
Arquivo 'cor.xlsx' carregado com sucesso!
Arquivo 'categorias.xlsx' carregado com sucesso!
Arquivo 'marcas.xlsx' carregado com sucesso!
Arquivo 'fornecedores.xlsx' carregado com sucesso!
Tipos de dados das colunas:
 Referências                         object
Código Produto - SKU                object
Status do Produto                    int64
Nome do Produto                     object
Categoria ID                         int64
Grade ID                             int64
Exibir produto na home              object
Valor atacado                      float64
Peso do Produto                     object
Tamanho do Produto                  object
Marca ID                             int64
Fornecedor ID                        int64
Código de Barras                     int64
Coleção                             object
Valor varejo                       float64
Área de Disponibilidade             object
Descrição do Produto                object
Cor      

,Referências,Código Produto - SKU,Status do Produto,Nome do Produto,Categoria ID,Grade ID,Exibir produto na home,Valor atacado,Peso do Produto,Tamanho do Produto,...,Coleção,Valor varejo,Área de Disponibilidade,Descrição do Produto,Cor,Imagens Vínculadas (Sim ou Não),Multiplicador Padrão,Quantidade Mínima por Pedido,Quantidade Máxima por Pedido,Forma de Limite
0,CAM1VER26TOY,CAM1VER26TOY-001,1,Camiseta oversized Kings Collab pixar Toy Stor...,3,21,Não,NaN,,,...,KINGS COLLAB PIXAR TOY STORY,NaN,{P},,1,Sim,1,6,999999,Limite por item da grade
6,CAM1VER26TOY,CAM1VER26TOY-006,1,Camiseta oversized Kings Collab pixar Toy Stor...,3,21,Não,NaN,,,...,KINGS COLLAB PIXAR TOY STORY,NaN,{P},,2,Sim,1,6,999999,Limite por item da grade
12,CAM2VER26TOY,CAM2VER26TOY-001,1,Camiseta oversized Kings Collab pixar Toy Stor...,3,21,Não,NaN,,,...,KINGS COLLAB PIXAR TOY STORY,NaN,{P},,1,Sim,1,6,999999,Limite por item da grade
18,CAM2VER26TOY,CAM2VER26TOY-006,1,Camiseta oversized Kings Collab pixar Toy Stor...,3,21,Não,NaN,,,...,KINGS COLLAB PIXAR TOY STORY,NaN,{P},,2,Sim,1,6,999999,Limite por item da grade
24,CAM3VER26TOY,CAM3VER26TOY-001,1,Camiseta oversized Kings Collab pixar Toy Stor...,3,21,Não,NaN,,,...,KINGS COLLAB PIXAR TOY STORY,NaN,{P},,1,Sim,1,6,999999,Limite por item da grade


Arquivo salvo como 'upload.xlsx' com sucesso!
